In [90]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [92]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-04 17:00:00+00:00


In [93]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [94]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-04 11:38:44,695 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 11:38:44,743 INFO: Initializing external client
2025-03-04 11:38:44,743 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 11:38:45,353 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [95]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.43s) 


In [96]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,252,0.0,2025-03-04 16:00:00+00:00
1,243,1.0,2025-03-04 16:00:00+00:00
2,224,3.0,2025-03-04 16:00:00+00:00
3,117,0.0,2025-03-04 16:00:00+00:00
4,69,0.0,2025-03-04 16:00:00+00:00
...,...,...,...
497,162,187.0,2025-03-04 17:00:00+00:00
498,196,1.0,2025-03-04 17:00:00+00:00
499,212,0.0,2025-03-04 17:00:00+00:00
500,246,101.0,2025-03-04 17:00:00+00:00


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  502 non-null    int32                  
 1   predicted_demand    502 non-null    float64                
 2   pickup_hour         502 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 9.9 KB


In [98]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 11:38:47,836 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 11:38:47,852 INFO: Initializing external client
2025-03-04 11:38:47,855 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 11:38:48,443 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Current UTC time: 2025-03-04 16:38:47.836785+00:00
Next hour: 2025-03-04 17:00:00+00:00
Querying for date range: 2025-03-04 to 2025-03-05
Filtering for hour: 2025-03-04 17:00
Found 251 records


In [99]:
now = datetime.now(timezone.utc)

In [100]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
251,226,2.0,2025-03-04 17:00:00+00:00
252,127,0.0,2025-03-04 17:00:00+00:00
253,68,168.0,2025-03-04 17:00:00+00:00
254,184,0.0,2025-03-04 17:00:00+00:00
255,155,1.0,2025-03-04 17:00:00+00:00
...,...,...,...
497,162,187.0,2025-03-04 17:00:00+00:00
498,196,1.0,2025-03-04 17:00:00+00:00
499,212,0.0,2025-03-04 17:00:00+00:00
500,246,101.0,2025-03-04 17:00:00+00:00


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502 entries, 0 to 501
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  502 non-null    int32                  
 1   predicted_demand    502 non-null    float64                
 2   pickup_hour         502 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 9.9 KB


In [102]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [103]:
dt = current_date.ceil('h')


In [104]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 11:38:50,045 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 11:38:50,053 INFO: Initializing external client
2025-03-04 11:38:50,053 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 11:38:50,664 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


In [105]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 


,pickup_location_id,predicted_demand,pickup_hour


In [106]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,252,0.0,2025-03-04 16:00:00+00:00
1,243,1.0,2025-03-04 16:00:00+00:00
2,224,3.0,2025-03-04 16:00:00+00:00
3,117,0.0,2025-03-04 16:00:00+00:00
4,69,0.0,2025-03-04 16:00:00+00:00
...,...,...,...
497,162,187.0,2025-03-04 17:00:00+00:00
498,196,1.0,2025-03-04 17:00:00+00:00
499,212,0.0,2025-03-04 17:00:00+00:00
500,246,101.0,2025-03-04 17:00:00+00:00


In [107]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [108]:
predictions = fetch_next_hour_predictions()

2025-03-04 11:38:56,797 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 11:38:56,797 INFO: Initializing external client
2025-03-04 11:38:56,797 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 11:38:57,375 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673
Current UTC time: 2025-03-04 16:38:56.797560+00:00
Next hour: 2025-03-04 17:00:00+00:00
Found 251 records


In [109]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-04 11:38:58,271 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 11:38:58,289 INFO: Initializing external client
2025-03-04 11:38:58,291 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 11:38:58,862 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215673


In [110]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.33s) 


In [111]:
df["pickup_hour"].max()

Timestamp('2025-03-04 17:00:00+0000', tz='Etc/UTC')

In [112]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-04 17:00:00+00:00


In [113]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
251,226,2.0,2025-03-04 17:00:00+00:00
252,127,0.0,2025-03-04 17:00:00+00:00
253,68,168.0,2025-03-04 17:00:00+00:00
254,184,0.0,2025-03-04 17:00:00+00:00
255,155,1.0,2025-03-04 17:00:00+00:00
...,...,...,...
497,162,187.0,2025-03-04 17:00:00+00:00
498,196,1.0,2025-03-04 17:00:00+00:00
499,212,0.0,2025-03-04 17:00:00+00:00
500,246,101.0,2025-03-04 17:00:00+00:00
